This notebook creates a basic neural network of arbitrary layer size to recognize digits in the MNIST dataset.


Data Source: http://yann.lecun.com/exdb/mnist/

Note: I give credit to Andrew Ng course on Coursera (at https://www.deeplearning.ai/), which is where I have learned to build neural networks. The structure of this notebook is similiar to the notebook's used in the course, but I have not copied anything from the course's notebooks (in accordance with the course license). The similarity with those notebooks is because that is the structure I learned from. 

Ideas for improvement
 - change cost function to output 0 if the prediction is correct, or 1 if it is wrong
 - standardize the dataset
 - create more detailed introduction to the data
 
 
 Misteps
 - didn't include the 1/m factor in cost function

In [78]:
import numpy as np

Import the data sets into python based on the method described here - https://rasbt.github.io/mlxtend/user_guide/data/loadlocal_mnist/

In [282]:
from mlxtend.data import loadlocal_mnist

X_train, y_train = loadlocal_mnist(
        images_path='train-images-idx3-ubyte', 
        labels_path='train-labels-idx1-ubyte')

X_test, y_test = loadlocal_mnist(
        images_path='t10k-images-idx3-ubyte', 
        labels_path='t10k-labels-idx1-ubyte')

#transposing the data so that the dimensions are features (784 pixels) x examples (60,000 images in training set)
X_train = X_train.T
X_test = X_test.T
y_train = np.squeeze(y_train).T
y_test = np.squeeze(y_test).T

print('X_train dims: %s x %s' % (X_train.shape[0], X_train.shape[1]))
print('y_train:'+ str(y_train.shape )) 
print('X_test dims: %s x %s' % (X_test.shape[0], X_test.shape[1]))
print('y_test:'+ str(y_test.shape )) 

print('\n1st row', y_train[5])

X_train dims: 784 x 60000
y_train:(60000,)
X_test dims: 784 x 10000
y_test:(10000,)

1st row 2


In [309]:
y1 = y_train-0
y1.shape
y2= m[0]-y_train
print(m[0][0,0:15])
print(y_train[0:15])
print(1/m[0].shape[1]*np.dot(y2,y2.T))

[2. 2. 5. 1. 2. 2. 3. 2. 4. 3. 2. 8. 3. 2. 5.]
[5 0 4 1 9 2 1 3 1 4 3 5 3 6 1]
[[14.4951]]


In [180]:
def initialize(layers_dims):
    '''
    Description
    This funciton initializes the parameters in the neural network with the W-weights initialized to small random
        numbers and the bias terms, b, intialized to zero
    
    Input
    layers_dim - the size of the neural network layers with the zero-th element being the number of examples
                - shape (1 x number of layers = L )
    
    Output
    parameters - a dictionary of the initialized parameters 
    
    '''
    
    np.random.seed(2) #sets the seed of the random number generator
    parameters = {}

    for l in range(1,len(layers_dims)):
        parameters['W' + str(l)] = np.random.randn(layers_dims[l], layers_dims[l-1]) * 0.1
        parameters['b' + str(l)] = np.zeros((layers_dims[l],1))
        
    return parameters
    

In [310]:
parameters = initialize([X_train.shape[0],9])

In [313]:
parameters['W1'].shape

(9, 784)

In [83]:
def forward_linear_activation(A_prev, W_curr, b_curr, activation = 'linear'):
    '''
    Description
    Calculates the linear combination of a single layer in the network as well as the activation layer
    
    Input
    A_prev - the activation values of the previous layer in the neural network -
    W_curr - the W parameter for the current layer
    b_curr - the b parameter in the current layer
    activation - determines the activation function performed on the linear combination Z_curr
    
    Output
    cache - a cache that stores the linear combination Z_curr and activation values A_curr
    A_curr - the sum of the linear combination of the A_prev and W_curr values and the b_curr vector
    '''
    
    Z_curr = np.dot(W_curr, A_prev) + b_curr
    
    if activation == 'linear':
        A_curr = Z_curr
        
    elif activation == 'relu':
        pass
    
    cache = [Z_curr, A_curr]
    
    return A_curr, cache

In [271]:
def forward_prop(X_train, parameters, layers_dims):
    '''
    Description
    Calculates the forward propogation steps of the neural network
    
    Input
    X_train   - the training examples
        shape >> (number of features, n, x number of examples, m )
    layers_dims - a list the dimensions of the neural networks layers
        shape   >> (1 x number of layers, L)
    
    
    Output
    A_L    - the activation values of the final layer in the network
    caches - caches of all of the Z and A values of the hidden layers which will be used in back prop
    '''
    
    caches = {}
    caches['A0'] = X_train
    
    for l in range(1, len(layers_dims)):
        A_curr, [caches['Z' + str(l)], caches['A' + str(l)]] = forward_linear_activation(caches['A'+str(l-1)], 
                                                                    parameters['W' + str(l)], parameters['b' + str(l)], 'linear')
        
    return A_curr, caches

In [277]:
for l in range(1,2):
    print(l)

1


In [85]:
X = np.random.rand(10,5); layers_dims=[10,1]

In [111]:
A, c = forward_prop(X_train, [784,9])

In [118]:
print(c['A1'].shape[1])

60000


In [158]:
def predict(A_L):
    '''
    Description
    Takes in the final values of the neural network and returns the predicted values 
    
    Inputs
    A_L    - the output of the final layer of the neural network
    
    
    Outputs
    Y_hat   - the predicted values - I know, not very descriptive... srry
    
    '''
    Y_hat = np.zeros((1, A_L.shape[1]) ) 
    
    for i in range(Y_hat.shape[1]):
        Y_hat[0][i] = np.argmax(A_L[:, i])

    return Y_hat


In [185]:
def computeCost(Y_hat, Y):
    '''
    Description
    Computes the cost function for the linear classifier
    
    
    Input
    Y_hat - the predicted values for Y output by the neural network - shape = (1 x # examples)
    Y     - the output examples that we use to train the algorithm  - shape = (1 x # examples)
    
    Ouptut
    cost  - the cost of the current iteration of the algorithm. In this case, the L2 difference is used without 
        regularization
    
    '''
    
    m = Y_hat.shape[1]
    
    cost = 1/m * np.dot( Y_hat - Y, (Y_hat - Y).T)
    
    return cost

In [314]:
def back_prop(parameters, X, Y_hat, Y, layers_dims, learning_rate = 0.01):
    '''
    Description
    Calculates the gradients of the parameters
    
    Inputs
    parameters - 
    Y_hat      - the predicted values
    Y          - the known  values
    
    Outputs
    
    
    '''
    l=1
    m = Y_hat.shape[1]

    
    print('here I am')
    
    
    dW = 2/m * np.dot(Y_hat - Y, X.T)
    #print("dw " + str(dW))
    db = 2/m * (Y_hat - Y)
    #print(dW)
 
    
#    for l in range(len(layers_dims), 1):
    parameters['W' + str(l)] = parameters['W' + str(l)] - 100
    parameters['b' + str(l)] = parameters['b' + str(l)] - learning_rate * db
    
    #print(parameters['W' + str(l)])
    
    return parameters

In [315]:
for l in range(2,0, -1):
    print(l)

2
1


In [317]:
def model(X_train, Y_train, layers_dims, iterations = 20, print_cost = True):
    '''
    Description
    Combines all aspects of the neural network into a single model

    Inputs
    
    
    Outputs
    '''
    
    parameters = initialize(layers_dims)
    #print('here I am')
    
    for i in range(iterations):

        A_L, caches = forward_prop(X_train, parameters, layers_dims)

        predictions = predict(A_L)

        cost = computeCost(predictions, Y_train)

        parameters = back_prop(parameters, X_train, predictions, Y_train, layers_dims, learning_rate = 0.01)
        
        if i % 10 == 0:
            print('Cost for ' + str(i) + '= ' + str(cost) ) 
        if i ==5 : 
            print(predictions)
    
    final_prediction = predict(A_L)
    
    return final_prediction, parameters

In [318]:
m = model(X_train, y_train, [X_train.shape[0],9] , print_cost = True)

here I am
Cost for 0= [[14.4951]]
here I am
here I am
here I am
here I am
here I am
[[2. 2. 5. ... 2. 2. 2.]]
here I am
here I am
here I am
here I am
here I am
Cost for 10= [[14.4951]]
here I am
here I am
here I am
here I am
here I am
here I am
here I am
here I am
here I am


In [195]:
print(m)

(array([[2., 2., 5., ..., 2., 2., 2.]]), {'W1': array([[-0.04167578, -0.00562668, -0.21361961, ..., -0.06168445,
         0.03213358, -0.09464469],
       [-0.05301394, -0.1259207 ,  0.16775441, ..., -0.03284246,
        -0.05623108,  0.01179136],
       [ 0.07386378, -0.15872956,  0.01532001, ..., -0.08428557,
         0.10040469,  0.00545832],
       ...,
       [ 0.24144681,  0.01110384, -0.01494575, ..., -0.0933672 ,
         0.06313131,  0.05855765],
       [ 0.05064269, -0.09990983, -0.03725472, ...,  0.08231032,
        -0.05483214,  0.00767053],
       [ 0.07695631, -0.02233986, -0.0742913 , ...,  0.01833128,
         0.01864871, -0.07748973]]), 'b1': array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])})
